## Template for Model Calibration 

In [2]:
import pandas as pd
import numpy as np
from geoEpic.io import CropCom, ieParm
from geoEpic.core import Workspace

import os
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

### load parameter files and set sensitive parameters

In [4]:
cropcom = CropCom.load('./parms/defaults')

cropcom.set_sensitive(['./parms/sensitivity/cropcom_CY.csv', 
                       './parms/sensitivity/cropcom_LAI.csv'], [1, 2])

# save default cropcom file in the model dir.
cropcom.save('./model')

##### Verify sensitive parameters

In [5]:
cropcom.prms

,ID,Parm,Parm.Description,Select,Min,Max,Category,Range
0,1,WA,Biomass-Energy Ratio (CO2 = 330ppm),1,20.000,50.000,"1,2,4","(20.0, 50.0)"
1,2,HI,Harvest Index,1,0.300,1.000,4,"(0.3, 1.0)"
4,5,DMLA,Maximum potential leaf area index,1,5.000,9.000,2,"(5.0, 9.0)"
5,6,DLAI,Fraction of growing season when leaf area decl...,1,0.600,1.000,2,"(0.6, 1.0)"
6,7,DLAP1,First point on optimal leaf area development c...,1,15.010,15.060,2,"(15.01, 15.06)"
7,8,DLAP2,Second point on optimal leaf area development ...,1,50.930,50.970,2,"(50.93, 50.97)"
8,9,RLAD,Leaf area index decline rate parameter,1,0.500,2.000,2,"(0.5, 2.0)"
9,10,RBMD,Biomass-energy ratio decline rate parameter,1,0.500,1.500,"1,2,4","(0.5, 1.5)"
11,12,GSI,Maximum Stomatal Conductance,1,0.005,0.015,"1,2,3,4,5","(0.005, 0.015)"
12,13,CAF,Critical Aeration Factor,1,0.800,0.900,"1,2,3,4,5","(0.8, 0.9)"


In [6]:
ieparm = ieParm.load('./parms/defaults')
ieparm.set_sensitive(['./parms/sensitivity/parm_CY.csv', './parms/sensitivity/parm_LAI.csv'])
ieparm.save('./model')

In [8]:
ieparm.prms

,ID,Parm,Parm.Description,Select,Min,Max,Category,Unit,Range
60,61,PARM1,Crop canopy,1,1.0000,2.00,"1,2,3,4,5",NaN,"(1.0, 2.0)"
70,71,PARM11,Moisture required for seed germination,1,-30.0000,-10.00,"1,2,4",NaN,"(-30.0, -10.0)"
87,88,PARM28,Acceptable plant N stress level,1,0.0000,1.00,"1,4",NaN,"(0.0, 1.0)"
109,110,PARM50,Technology annual rate coefficient,1,0.0000,0.01,"4,5",NaN,"(0.0, 0.01)"
113,114,PARM54,Exponential coefficient in potential water use...,1,2.5000,7.50,"1,2,4,5",NaN,"(2.5, 7.5)"
114,115,PARM55,Coefficient used in allocating root growth,1,0.0000,1.00,"1,2,4",NaN,"(0.0, 1.0)"
115,116,PARM56,Exponential coefficient in root growth distrib...,1,5.0000,10.00,"1,2,3",NaN,"(5.0, 10.0)"
118,119,PARM59,Soil water value to delay tillage,1,0.9700,1.00,"1,2,4,5",NaN,"(0.97, 1.0)"
121,122,PARM62,Exponential coefficient regulates upward N mov...,1,0.2000,2.00,"1,2,4",NaN,"(0.2, 2.0)"
122,123,PARM63,Upper limit of N concentration in percolating ...,1,100.0000,10000.00,"1,2,4,5",NaN,"(100.0, 10000.0)"


### Define a fitness function

- fitness function is called after each workspace run, to get the objective of optimization 

In [ ]:
exp = Workspace('./config.yml')


@exp.logger
def lai_error(site):
    print(site.outputs['ACY'])
    return {'error': site.site_id}


### Make pygmo problem


In [ ]:
import pygmo as pg

problem = MakeProblem(fitness, cropcom, ieparm)

### Run Calibration
- Choose necessary settings and optimization algorithm
- Follow this link for [pygmo docs](https://esa.github.io/pygmo2/overview.html)

In [ ]:
algo = pg.algorithm(pg.pso(gen = 4))
algo.set_verbosity(1) 
pop = pg.population(pg.problem(problem), 30)
pop = algo.evolve(pop) 